In [55]:
import boto3
from io import StringIO
import pandas as pd
import numpy as np
import json

In [ ]:
s3 = boto3.client('s3')
bucket_name = 'mathcat-bucket'
file_key = 'DATA_preprocessing/Unsaved/2024/03/06/baba44b9-9567-4da7-872e-35cde599f185.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_key)
data = obj['Body'].read().decode('utf-8')
data_io = StringIO(data)

In [61]:
final_5g = pd.read_csv(data_io)
df5 = final_5g[['userid', 'quizcode', 'correct']]
df5['correct'] = df5['correct'].replace({'O': 1, 'X': 0})
# df5['Correct'] = df5['Correct'].replace({'O': 1, 'X': 0})
df5.columns = ['UserID', 'QuizCode', 'Correct']

/var/folders/81/248083pd4gs_dcbfd1cfps280000gn/T/ipykernel_55575/2204800672.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Correct'] = df5['Correct'].replace({'O': 1, 'X': 0})


In [62]:
# 500명 이상이 푼 문제만 추출
quiz_filter = df5.groupby(['QuizCode'])['UserID'].nunique().reset_index()
quiz_filter = quiz_filter[quiz_filter['UserID'] >= 500]['QuizCode']

# 20문제 이상 푼 사용자만 추출
user_filter = df5.groupby(['UserID'])['QuizCode'].nunique().reset_index()
user_filter = user_filter[user_filter['QuizCode'] >= 20]['UserID']

# 최종 테이블
df5_filter = df5[df5['QuizCode'].isin(quiz_filter) & df5['UserID'].isin(user_filter)]

In [63]:
user_quiz_dict = {}

for index, row in df5.iterrows():
    user_id = row['UserID']
    quiz_code = f'quiz{row["QuizCode"]}'
    correct = row['Correct']

    if user_id not in user_quiz_dict:
        user_quiz_dict[user_id] = {}
    user_quiz_dict[user_id][quiz_code] = correct

In [64]:
with open('test1.jsonlines', 'w') as outfile:
    for user, quiz in user_quiz_dict.items():
        json.dump({"subject_id": user, "responses": quiz}, outfile)
        outfile.write('\n')

In [ ]:
!python ./irt_pipeline/py-irt/cli.py train 2pl test1.jsonlines test-2pl/ --lr 0.1 --epochs 2000

In [65]:
import json

file_path = './data/best_parameters_filter.json'

with open(file_path, 'r') as f:
    data = json.load(f)

quiz_code = list(data["item_ids"].values())

print(quiz_code)

['quiz30064595', 'quiz30064596', 'quiz30064597', 'quiz30064598', 'quiz30064599', 'quiz30064605', 'quiz30064606', 'quiz30064607', 'quiz30064608', 'quiz30064609', 'quiz30064166', 'quiz30064167', 'quiz30064168', 'quiz30064170', 'quiz30064171', 'quiz30064172', 'quiz30064173', 'quiz30064174', 'quiz30064175', 'quiz30064177', 'quiz30064178', 'quiz30064180', 'quiz30064181', 'quiz30064182', 'quiz30064183', 'quiz30064184', 'quiz30062297', 'quiz30062294', 'quiz30062293', 'quiz30062303', 'quiz30062302', 'quiz30062313', 'quiz30062315', 'quiz30062317', 'quiz30062327', 'quiz30062291', 'quiz30062295', 'quiz30062298', 'quiz30062300', 'quiz30062301', 'quiz30062304', 'quiz30062305', 'quiz30062306', 'quiz30062307', 'quiz30062308', 'quiz30062309', 'quiz30062310', 'quiz30062311', 'quiz30062312', 'quiz30062318', 'quiz30062319', 'quiz30062320', 'quiz30062321', 'quiz30062323', 'quiz30062324', 'quiz30062325', 'quiz30062326', 'quiz30062328', 'quiz30062329', 'quiz30062330', 'quiz30067093', 'quiz30067094', 'quiz30

In [66]:
disc = data['disc']

In [67]:
diff = data['diff']